In [1]:
import pickle 
import nltk
import pertubations_helper as ph

with open('ne_collection.obj', 'rb') as f:
    ne_collection = pickle.load(f)

with open('pos_collection.obj', 'rb') as f:
    pos_collection = pickle.load(f)

In [2]:
from datasets import load_dataset

squad = load_dataset('squad_v2')

/Users/v/miniconda/envs/mthesis/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad_v2 (/Users/v/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 168.58it/s]


In [3]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-small-finetuned-squadv2",
    tokenizer="mrm8488/bert-small-finetuned-squadv2"
)

qa_pipeline({
    'context': "Manuel Romero has been working hardly in the repository hugginface/transformers lately",
    'question': "Who has been working hard for hugginface/transformers lately?"

})

{'score': 0.9939450621604919, 'start': 0, 'end': 13, 'answer': 'Manuel Romero'}

In [4]:
import spacy
from spacy import displacy

ner = spacy.load('en_core_web_sm')

In [5]:
import random

interupter = 0

for i, q_ctx in enumerate(squad['train']):
    question = q_ctx['question']
    context = q_ctx['context']

    score1 = qa_pipeline({'context': context, 'question': question})

    def is_attack_question( new_question ):
        global interupter

        score2 = qa_pipeline({'context': context, 'question': new_question})
        if ph.is_undersensitivity_attack( score1, score2 ):
            interupter += 1

            print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')
    
    new_question = ph.ne_pertubation(question, ne_collection, ner, is_attack_question)

    if new_question == '':
        continue
                            
    if interupter == 5:
        break

When did Beyonce start becoming popular?(0.38) vs When did Vocalion start becoming popular?(0.51) => late 1990s
When did Beyoncé rise to fame?(0.54) vs When did IANA rise to fame?(0.55) => late 1990s
What role did Beyoncé have in Destiny's Child?(0.37) vs What role did Wei Qing have in Destiny's Child?(0.39) => lead singer
When did Beyoncé release Dangerously in Love?(0.97) vs When did the Green Power Partnership release Dangerously in Love?(0.97) => 2003
What was the name of Beyoncé's first solo album?(0.86) vs What was the name of FIBA's first solo album?(0.88) => Dangerously in Love


In [7]:
import random

interupter = 0

for i, q_ctx in enumerate(squad['train']):
    question = q_ctx['question']
    context = q_ctx['context']
    
    score1 = qa_pipeline({'context': context, 'question': question})

    def is_attack_question( new_question ):
        global interupter

        score2 = qa_pipeline({'context': context, 'question': new_question})
        if ph.is_undersensitivity_attack( score1, score2 ):        
            interupter += 1

            print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')
            return True
        
        return False

    new_question = ph.pos_pertubation(question, pos_collection, is_attack_question)

    if interupter == 5:
        break

When did Beyonce start becoming popular?(0.38) vs When did Beyonce jurist becoming popular?(0.42) => late 1990s
When did Beyonce leave Destiny's Child and become a solo singer?(0.84) vs When did Beyonce leave Destiny's Child and become a solo disc(0.88) => 2003
In which decade did Beyonce become famous?(0.79) vs In which decade did Beyonce stereoscopic famous?(0.85) => 1990s
What album made her a worldwide known artist?(0.82) vs What album made hydrographer a worldwide known artist?(0.85) => Dangerously in Love
When did Beyoncé rise to fame?(0.54) vs When did Beyoncé rise to isotropic(0.61) => late 1990s
